In [511]:
import os
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt

PassengerId : 각 승객의 고유 번호로 의미 없음

Survived : 생존 여부 (종속 변수)

Pclass : 티켓의 승선권 클래스 - 승객의 사회적, 경제적 지위를 알 수 있다. 클래스에 따라 좀 더 생존에 용이한 구조이지 않을까?

1st = Upper
2nd = Middle
3rd = Lower
Name : 이름 - 이름으로 딱히 의미가 있을까? 큰 의미 없을 듯.

Sex : 성별 - 성별에 따라 먼저 구조될 수도, 생존에 용이할 수 있을 듯하다.

Age : 나이 - 각 세대별(10대, 20대 또는 아이, 청소년, 성인 등)로 차이가 있을 수 있다.

SibSp : 동반한 Sibling(형제자매)와 Spouse(배우자)의 수 - 동반 여행자의 수에 따라 생존 차이가 있을 수도 있다.

Parch : 동반한 Parent(부모) Child(자식)의 수 - 위와 동일

Ticket : 티켓의 고유넘버 - 티켓 자체로 생존여부를 알 수는 없을 듯하다.

Fare : 티켓의 요금 - 비싼 요금의 티켓과 생존 여부에 상관이 있을 수도 있다. 하지만 Pclass가 이 변수를 categorical하게 만든 변수가 아닐까?

Cabin : 객실 번호 - 어떤 객실 번호 범위는 좀 더 갑판 등의 생존에 용이한 위치에 있을 수도 있을 듯하다.

Embarked : 승선한 항 - 큰 의미 없을 듯하다.

C = Cherbourg
Q = Queenstown
S = Southampton

In [512]:
# 데이터 불러오기
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")

In [513]:
# 훈련 데이터의 상위 7개 데이터 펼쳐보기
train_data.head(7)

In [514]:
# 테스트 데이터의 상위 7개 데이터 펼쳐보기
test_data.head(7)

In [515]:
# 훈련 데이터의 NULL 값 확인
train_data.info()

In [516]:
# 테스트 데이터의 NULL 정보 확인
# Age 값과 Cabin 값의 NULL 정보를 확인할 수 있음
test_data.info()

> Survived

In [517]:
#  38 퍼센트의 사람이 생존했다는 것을 훈련 데이터에서 확인할 수 있다.
train_data['Survived'].value_counts(normalize=True)

In [518]:
# 그래프로 확인할 수 있는 생존 데이터
sns.countplot(train_data['Survived'])

> P CLASS

In [519]:
# 훈련 데이터에서 생존자들의 P class의 숫자가 낮음에 따라 생존율이 낮다는 것을 확인할 수 있다.
train_data['Survived'].groupby(train_data['Pclass']).mean()

![](https://static1.squarespace.com/static/5006453fe4b09ef2252ba068/t/5090b249e4b047ba54dfd258/1351660113175/TItanic-Survival-Infographic.jpg?format=1500w)

제 3 클래스의 좌석부터 가라 앉는 것을 볼 수 있고, 보통 지하층에 위치하는 것을 볼 수 있음

> NAME

In [520]:
# 정규식을 사용하여, train_data에서 이름을 추출
train_data['Initial'] = train_data.Name.str.extract('([A-Za-z]+)\.')
test_data['Initial'] = test_data.Name.str.extract('([A-Za-z]+)\.')

In [521]:
# 훈련 데이터에서 추출한 이니셜의 개수 출력
train_data['Initial'].value_counts(normalize=False)

In [522]:
# 훈련 데이터의 생존자에서 확인할 수 있는 이니셜의 평균치
train_data['Survived'].groupby(train_data['Initial']).mean()

In [523]:
# 훈련 데이터의 성별과 추출한 이름(성)에 대한 정보 나열
pd.crosstab(train_data['Initial'], train_data['Sex'])

In [524]:
# 위의 시각화한 자료를 토대로 이니셜을 Miss, Mr, Mrs, 그 밖에는 Other로 대체
# 그리고 initial의 평균 값을 도출
train_data['Initial'].replace(['Mlle','Mme','Ms','Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don','Dona'],['Miss','Miss','Miss','Mr','Mr','Mrs','Mrs','Other','Other','Other','Mr','Mr','Mr','Mr'],inplace=True)

> Sex

In [525]:
# 훈련 데이터의 생존자 중,성별의 비율
# 이건 당연하게도, 여성과 아이들이 먼저라는 타이타닉 항해사의 말을 참고해보았을 때 맞는 결과이다.
train_data['Survived'].groupby(train_data['Sex']).mean()

> Age

In [526]:
# 각 이름 성에 따른 평균 값 확인
train_data.groupby('Initial').mean()

In [527]:
# 훈련 데이터에서 Age의 평균 값을 도출하기
train_data.loc[(train_data.Age.isnull())&(train_data.Initial=='Mr'),'Age'] = 33
train_data.loc[(train_data.Age.isnull())&(train_data.Initial=='Miss'),'Age'] = 22
train_data.loc[(train_data.Age.isnull())&(train_data.Initial=='Master'),'Age'] = 5
train_data.loc[(train_data.Age.isnull())&(train_data.Initial=='Mrs'),'Age'] = 36
train_data.loc[(train_data.Age.isnull())&(train_data.Initial=='Other'),'Age'] = 46

> Embarked

In [528]:
# 생존자 중, Embarked의 분포도 확인
train_data['Survived'].groupby(train_data['Embarked']).mean()

In [529]:
# Embarked의 분포도 확인
train_data['Embarked'].value_counts()

In [530]:
# 2개의 NULL 데이터를 많은 데이터인 S로 적용시킨다.
train_data['Embarked'].fillna('S',inplace=True)

> SibSp

In [531]:
# 생존자 중, 동반 인원이 있는 사람들의 생존 평균
train_data['Survived'].groupby(train_data['SibSp']).mean()

In [532]:
# 혼자 탔다고 생존율이 높고 한 것 아니니, 그리 좋은 양분은 될 수 없음
train_data['SibSp'].value_counts()

> Parch

In [533]:
# 비교적 고르게 분포한 데이터를 볼 수 있는데, 그다지 효율적으로 쓰이지는 않을 것 같은 데이터이다.
train_data['Survived'].groupby(train_data['Parch']).mean()

In [534]:
# 처음에는 혼자인 사람들의 생존율이 높을거라고 생각을 했지만, 위의 데이터를 확인했을 때 맞지 않는 것을 확인할 수 있었다. 그다지 효율적으로 쓰이지는 않을 것 같은 데이터이다.
train_data['Parch'].value_counts()

> Fare

In [535]:
# 티겟을 범위를 5등분
pd.qcut(train_data['Fare'], 5).value_counts()

In [536]:
# 티켓의 값에 따라 생존율이 올라가는 것을 볼 수 있다.
train_data['Survived'].groupby(pd.qcut(train_data['Fare'], 5)).mean()

In [537]:
# 티겟의 값을 5등분하여 P Class에 대한 분포도를 확인할 수 있었다.
pd.crosstab(pd.qcut(train_data['Fare'], 5), columns=train_data['Pclass'])

> Cabin

In [538]:
# Cabin의 데이터를 효율적으로 보기 위해서 앞 글자를 따온다.
train_data['Cabin_Initial'] = train_data['Cabin'].apply(lambda x: str(x)[0])

In [539]:
# 700개 다달하는 NULL 값을 가지고 있지만, 결측치가 너무 많아서 사용하기 힘들다.
train_data['Cabin_Initial'].value_counts()

> 작업 시작

In [540]:
# 변화된 데이터 값 확인
train_data.info()

In [541]:
# 변화된 테스트 데이터의 확인
test_data.info()

In [542]:
# 테스트 데이터에도 똑같이 AGE의 평균 값을 넣어준다
test_data['Initial'].replace(['Mlle','Mme','Ms','Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don','Dona'],['Miss','Miss','Miss','Mr','Mr','Mrs','Mrs','Other','Other','Other','Mr','Mr','Mr','Mr'],inplace=True)
test_data.groupby('Initial').mean()

In [543]:
test_data.loc[(test_data.Age.isnull())&(test_data.Initial=='Mr'),'Age'] = 32
test_data.loc[(test_data.Age.isnull())&(test_data.Initial=='Miss'),'Age'] = 22
test_data.loc[(test_data.Age.isnull())&(test_data.Initial=='Master'),'Age'] = 7
test_data.loc[(test_data.Age.isnull())&(test_data.Initial=='Mrs'),'Age'] = 39
test_data.loc[(test_data.Age.isnull())&(test_data.Initial=='Other'),'Age'] = 43

# Fare에 값에 평균 값을 넣어준다.
test_data.loc[test_data.Fare.isnull(),'Fare'] = test_data['Fare'].mean()

In [544]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

y = train_data["Survived"]

features = ["Pclass", "Sex", "Age", "Fare"] # 특성 뽑기
x = pd.get_dummies(train_data[features]) # 훈련 데이터에 특성 적용해서 뽑아내기
x_test = pd.get_dummies(test_data[features]) # 테스트 데이터에 특성 적용해서 뽑아내기
model = RandomForestClassifier(random_state=1)

In [545]:
# 랜덤 포레스트 + 그리드 서치
param_grid = [{'n_estimators': range(5, 100, 10), 'max_depth': range(3, 5)}]
gs = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=5, n_jobs=-1, refit=True)
gs.fit(x, y)

print('베스트 하이퍼 파라미터 : {0}'.format(gs.best_params_))
# 베스트 하이퍼 파라미터 : {'max_depth': 4, 'n_estimators': 25}

# 최적화 된 모델을 추출한다.
best_model = gs.best_estimator_
# 도출된 값을 통해 모델을 예측한다
pred = best_model.predict(x_test)

# CSV 파일 출력단
output = pd.DataFrame({'PassengerId' : test_data.PassengerId, 'Survived': pred})
output.to_csv("RF.csv", index = False)
print("success")